In [ ]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import sqlite3
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from unidecode import unidecode
import time

with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)
#consumer key, consumer secret, access token, access secret.
ckey=creds['CONSUMER_KEY']
csecret=creds['CONSUMER_SECRET']
atoken=creds['ACCESS_TOKEN']
asecret=creds['ACCESS_SECRET']

conn = sqlite3.connect('twitter_lol_TB.db')
c = conn.cursor()

def create_table():
    try:
        c.execute("CREATE TABLE IF NOT EXISTS sentiment(unix REAL, tweet TEXT, sentiment REAL)")
        c.execute("CREATE INDEX fast_unix ON sentiment(unix)")
        c.execute("CREATE INDEX fast_tweet ON sentiment(tweet)")
        c.execute("CREATE INDEX fast_sentiment ON sentiment(sentiment)")
        conn.commit()
    except Exception as e:
        print(str(e))
create_table()

class listener(StreamListener):

    def on_data(self, data):
        try:
            data = json.loads(data)
            tweet = unidecode(data['text'])
            time_ms = data['timestamp_ms']
            analysis = TextBlob(tweet)
            if analysis.detect_language() == 'en':
                analysis = analysis
            else:
                analysis = analysis.translate(to='en')
                print(analysis)
            sentiment = analysis.sentiment.polarity
            print(time_ms, tweet, sentiment)
            c.execute("INSERT INTO sentiment (unix, tweet, sentiment) VALUES (?, ?, ?)",
                  (time_ms, tweet, sentiment))
            conn.commit()

        except KeyError as e:
            print(str(e))
        return(True)

    def on_error(self, status):
        print(status)

while True:

    try:
        auth = OAuthHandler(ckey, csecret)
        auth.set_access_token(atoken, asecret)
        twitterStream = Stream(auth, listener())
        twitterStream.filter(track=["leagueoflegends", "leaguepreseason", "aphelios", "season10", "sett"])
    except Exception as e:
        print(str(e))
        time.sleep(2)

index fast_unix already exists
1576524245928 RT @katekeycosplay: Really thanks for support my new cosplay! I did the same years ago but this new version is STUNNING [?][?][?][?] https://t.... 0.20136363636363636
1576524246446 RT @Foreversexrol: Animation by counter play,Patreon:EIPRIL
#Leagueoflegends #Rule34 #hentaiespanol #Sexrol https://t.co/NkUdkqfQen 0.0
RT @Hanna__Lindholm: We have seen a lot of swine. But I never thought we would see a top name in LO Hannah and giggle at an innocent dad ...
1576524246504 RT @Hanna__Lindholm: Mycket svineri har vi sett. Men jag trodde aldrig vi skulle se ett toppnamn i LO hana och fnittra at en oskyldig pappa... 0.5
1576524248843 RT @victorli_ran: when enemy Sett about to through a 6k HP Sion at you 
#ArtofLegends #LeagueOfLegends #jojosbizarreadventure https://t.co/... 0.0
1576524253990 RT @Luscious_Net: #LeagueOfLegends Tournament - Match #1 (Round 1)

#WhoWouldYouRather? #Leona  #Zyra 

(Poll in comments section) https:/... -0.2
1576524260629 